In [1]:
import numpy as np
import pymc as pm
import arviz as az

from scipy.stats import binom

In [2]:
class BasketTrialSimulator:
    def __init__(self, K, alpha, num_sim, Ni, Ni1, q0, q1, Qf, draws, tune, chains, pbar):
        self.K = K
        self.alpha = alpha
        self.num_sim = num_sim
        self.Ni = Ni
        self.Ni1 = Ni1
        self.q0 = q0
        self.q1 = q1
        self.Qf = Qf
        self.draws = draws
        self.tune = tune
        self.chains = chains
        self.pbar = pbar

        # Initialize matrices
        self.nik = np.zeros((2, self.K), dtype=int)
        self.rik = np.zeros((2, self.K))
        self.nik[0,:] = self.Ni1
        self.p0 = np.full(self.K, self.q0)
        self.posterior_ind = np.zeros((self.num_sim, self.K))
        self.Q_calibrated = None

    def beta_binomial_model(self, n, Y):
        assert len(n) == len(Y)
        K = len(n)
        with pm.Model() as model:
            α = pm.Gamma('alpha', alpha=2, beta=0.5)
            β = pm.Gamma('beta', alpha=2, beta=0.5)
            θ = pm.Beta('mu', alpha=α, beta=β, shape=K)
            y = pm.Binomial('y', n=n, p=θ, observed=Y, shape=K)
        return model

    def generate_data(self, n, p):
        return binom.rvs(n=n, p=p)

    def calculate_posterior(self, trace, q):
        stacked = az.extract(trace)        
        basket_probs = stacked.mu.values
        posterior = np.zeros(len(basket_probs))
        for k in range(len(basket_probs)):
            posterior[k] = np.mean(basket_probs[k, :] > q)
        return posterior

    def calibrate(self):
        for sim in range(self.num_sim):
            print(sim)
            
            n = self.nik[0, :]
            self.rik[0, :] = self.generate_data(n=n, p=self.p0)  # generate response data

            Y = self.rik[0,:]
            num_baskets = len(n)

            with self.beta_binomial_model(n=n, Y=Y) as model:
                trace = pm.sample(self.draws, tune=self.tune, chains=self.chains, progressbar=self.pbar)

            ## Interim analysis:
            midpoint = (self.q0 + self.q1) / 2
            posterior = self.calculate_posterior(trace, midpoint)

            ## Futility stop:                    
            stage2_stop = np.where(posterior < self.Qf)[0]
            stage2_cont = np.where(posterior >= self.Qf)[0]
            self.nik[1, stage2_cont] = self.Ni - self.Ni1

            # Store posterior of success from interim analysis            
            self.posterior_ind[sim, stage2_stop] = posterior[stage2_stop]

            # Stage 2:            
            if len(stage2_cont) > 0:
                
                # generate response data                
                self.rik[1, stage2_cont] = self.generate_data(n=self.nik[1, stage2_cont], p=self.p0[stage2_cont])
                ni = np.sum(self.nik[:, stage2_cont], axis=0)
                Y = np.sum(self.rik[:, stage2_cont], axis=0)
                num_baskets = len(ni)

                with self.beta_binomial_model(n=ni, Y=Y) as model2:
                    trace2 = pm.sample(self.draws, tune=self.tune, chains=self.chains, progressbar=self.pbar)

                # Final decision
                posterior = self.calculate_posterior(trace2, self.q0)

                self.posterior_ind[sim, stage2_cont] = posterior

        self.Q_calibrated = np.quantile(self.posterior_ind, 1-self.alpha)

    def simulate(self):
        pass  # Simulation method will be implemented here.

In [3]:
# Set simulation settings and parameters
K = 6 # number of indications
alpha = 0.1 # significance level for the test
num_sim = 10 # number of simulations per simulation setting
Ni = 24 # maximum of total sample size for each indication group
Ni1 = 14 # stage-one sample size for each indication group

# Set null and target response rates
q0 = 0.2 # standard of care (null) response rate
q1 = 0.4 # target response rate
Qf = 0.05 # probability cut-off for interim analysis

draws = 1000
tune = 1000
chains = 1
pbar = False

In [4]:
trial = BasketTrialSimulator(K, alpha, num_sim, Ni, Ni1, q0, q1, Qf, draws, tune, chains, pbar)

In [5]:
trial.calibrate()

0


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


2


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


8


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


9


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


In [6]:
trial.Q_calibrated

0.8542